## Documents

LangchainにはDocumentがある。Documentは次の属性を持つ。

- page_content:contentのstring
- metadata: dictionary
- id : documentのid

In [1]:
from langchain_core.documents import Document


In [2]:
documents = [
    Document(
        page_content="塩ラーメン：魚介を使ったあっさり系",
        metadata = {"source": "メニュー"}
    ),
    Document(
        page_content="醤油ラーメン：豚骨べーすのがっつり系",
        metadata = {"source": "メニュー"}
    )
]

## PDFの読み込み
- ./pdfに任意のPDFを配置

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../pdf/main.pdf")

document = loader.load()

In [9]:
# pdfのページごとに読み込む
print(len(document))

32


## 分割
- 1ページだとデータが大きすぎる
- text splitterで分割
- 文字数ごと

### 引数
- chunk_dize:チャンクごとの文字数
- chunk_overlap:何文字重ねるか？

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap = 600,
    add_start_index = True
)

splits_data = text_splitter.split_documents(document)

In [13]:
print(len(splits_data))

157


## 埋め込みベクトル化(embedding)
- huggingfaceのモデルでベクトル化を行う

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

emb_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
) 

In [20]:
# ベクトル化した一部を出力
emb_model.embed_query(splits_data[0].page_content)[:10]

[0.026626722887158394,
 0.056824635714292526,
 0.008822723291814327,
 -0.05688439682126045,
 -0.0035381163470447063,
 0.005480628926306963,
 -0.03550500422716141,
 0.008347484283149242,
 -0.02197660319507122,
 -0.01593012548983097]

### ベクトルの保存

In [33]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(emb_model)

# ドキュメントのindex

ids = vector_store.add_documents(documents=splits_data)

C:\Users\kijuh\AppData\Local\Temp\ipykernel_35164\4141838185.py:7: RuntimeWarning: coroutine 'InMemoryVectorStore.aadd_documents' was never awaited
  ids = vector_store.add_documents(documents=splits_data)


In [34]:
print(len(ids))

157


In [45]:
results = vector_store.similarity_search(
    "研究背景を教えて"
)

In [50]:
# scoreも必要な場合
results = vector_store.similarity_search_with_score(
    "研究背景を教えて"
)
doc,score = results[0]
print(score)

0.14720373082056024


In [52]:
## retriver

retriver = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

data = retriver.batch(
    [
        "計算はどれくらい早くなった?",
        "何のGPUを使っている？"
    ]
)